In [1]:
%load_ext autoreload
%autoreload 2

First, let get some utilities from brainiac_temporal

In [2]:
%cd /workdir

/workdir


In [3]:
from sota_paper_implementation.convertors.d2g22tgt import convert_d2g2_sample_to_tgt
from brainiac_temporal.metrics.utility import link_forecasting_metric
import numpy as np
from pathlib import Path
import torch

Now, into D2G2 itself!

In [5]:
%cd sota_paper_implementation/D2G2

/workdir/sota_paper_implementation/D2G2


## Get the dataset

In [8]:
data_path = Path("dataset/insecta/")
originals = convert_d2g2_sample_to_tgt(data_path, "")

# --------- load data
adj = np.load(data_path / Path("adj.npy"), allow_pickle=True)
features = np.load(data_path / Path("feature.npy"), allow_pickle=True)

# --------- after loading dataset
# to torch
adj = torch.from_numpy(adj).float()
features = torch.from_numpy(features).float()
dataset = torch.utils.data.TensorDataset(adj, features)

(1, 39, 165, 165)
(1, 39, 165, 1)


## Train the model

In [9]:
from model import *
from trainer import *

**You probably want to change 2 things:**

`device = torch.device('cpu')` to cuda:0 if you have GPU access
and
`epochs=1` to a value that will actually train the model ;) Keep at 1 for fast testing

In [6]:
# device = torch.device('cuda:0')
device = torch.device("cpu")

epochs = 1

In [11]:
batch_size = 8
z_dim = 32
genr_batch_size = 100
f_dim= 256

seq_len = adj.size(1)
max_num_nodes = adj.size(2)
feature_dim = features.size(3)

loader = torch.utils.data.DataLoader(
    dataset, batch_size=batch_size, shuffle=True, num_workers=4
)

d2g2 = D2G2(
    f_dim=f_dim,
    z_dim=z_dim,
    batch_size=batch_size,
    seq_len=seq_len,
    factorised=True,
    device=device,
    graphs=seq_len,
    feature_dim=feature_dim,
    max_num_nodes=max_num_nodes,
)

fix_f = torch.rand(f_dim, device=device)
fix_f = fix_f.expand(genr_batch_size, seq_len, f_dim)
test_f = fix_f

trainer = Trainer(
    d2g2,
    loader,
    test_f,
    epochs=epochs,
    learning_rate=0.0002,
    device=device,
    max_num_nodes=max_num_nodes,
    genr_batch_size=genr_batch_size,
    original_full_adj=adj,
)


In [14]:
import os

os.makedirs("output", exist_ok=True)
trainer.train_model()
trainer.sample_graphs()


Running Epoch : 1


1it [00:01,  1.46s/it]


Epoch 1 : Average Loss: 183957094400.0 Edge Loss 6.967780590057373 Node Loss 76105310208.0 KL of f : 88.32738494873047 KL of z : 107851792384.0 Bursty_Coeff: 0.0 Temporal_Efficiency: 0.0 Degree_Centrality 0.0 original_edges_total 78.0 recon_edges_total 534501.0
Training is complete
from sample_frames: edge and node:  torch.Size([100, 39, 32]) torch.Size([100, 39, 32])
from sample_frames: test_f_expand.shape:  torch.Size([100, 39, 256])
torch.Size([100, 39, 320]) torch.Size([100, 39, 320])
from sample_frames: recon.shape:  torch.Size([100, 39, 165, 165]) torch.Size([100, 39, 165, 1])


You samples are now available in the output directory!

D2G2 has hardcoded the sample path to `output/adj_metro_fix_z.npy` `output/feature_metro_fix_z.npy`.

we recommand moving it, to avoid it being overridden at the next run.

They can be converted to the torch geometric temporal:

In [15]:
samples = convert_d2g2_sample_to_tgt("output/", "_metro_fix_z")

(100, 39, 165, 165)
(100, 39, 165, 1)
